In [67]:
import pandas as pd 
from functools import reduce

In [36]:
df = pd.read_csv(
    '../data/running-records.csv'
)

In [37]:
df.head()

,Rank,Time,Name,Country,Date of Birth,Place,City,Date,Gender,Event
0,1,2:00:35,Kelvin Kiptum,KEN,02.12.99,1,Chicago,08.10.2023,Men,Marathon
1,2,2:01:09,Eliud Kipchoge,KEN,05.11.84,1,Berlin,25.09.2022,Men,Marathon
2,3,2:01:25,Kelvin Kiptum,KEN,02.12.99,1,London,23.04.2023,Men,Marathon
3,4,2:01:39,Eliud Kipchoge,KEN,05.11.84,1,Berlin,16.09.2018,Men,Marathon
4,5,2:01:41,Kenenisa Bekele,ETH,13.06.82,1,Berlin,29.09.2019,Men,Marathon


In [38]:
df['Datetime'] = df['Time'].apply(pd.to_datetime, errors='coerce')
df['City'] = df['City'].replace('New York City', 'New York')
df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True)

/opt/homebrew/lib/python3.11/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '25.09.2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
/opt/homebrew/lib/python3.11/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '23.04.2023' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
/opt/homebrew/lib/python3.11/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '16.09.2018' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
/opt/homebrew/lib/python3.11/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '29.09.2019' in DD/MM/YYYY format. Provide format or specif

In [39]:
marathons = df.loc[
    (df['Event'] == 'Marathon') & 
    (df['City'].isin(['Tokyo', 'Boston', 'London', 'Berlin', 'Chicago', 'New York']))
]

In [20]:
idx = marathons.groupby(['City', 'Gender'])['Datetime'].idxmin()
best_times = marathons.loc[idx][['Time', 'Name', 'Country', 'City', 'Gender', 'Date']]
best_times

,Time,Name,Country,City,Gender,Date
1,2:01:09,Eliud Kipchoge,KEN,Berlin,Men,2022-09-25
18191,2:11:53,Tigist Assefa,ETH,Berlin,Women,2023-09-24
16,2:03:02,Geoffrey Mutai,KEN,Boston,Men,2011-04-18
18275,2:18:57,Rita Sitienei Jeptoo,KEN,Boston,Women,2014-04-21
0,2:00:35,Kelvin Kiptum,KEN,Chicago,Men,2023-08-10
18192,2:13:44,Sifan Hassan,NED,Chicago,Women,2023-08-10
2,2:01:25,Kelvin Kiptum,KEN,London,Men,2023-04-23
18196,2:15:25,Paula Radcliffe,GBR,London,Women,2003-04-13
199,2:05:06,Geoffrey Mutai,KEN,New York,Men,2011-06-11
18631,2:22:31,Margaret Okayo,KEN,New York,Women,2003-02-11


In [21]:
best_times.to_json(
    '../data/best_times.json', 
    orient='records',
    indent=2
)

In [33]:
idx = marathons.groupby(['City', 'Gender'])['Date'].idxmax()
latest_times = marathons.loc[idx][['Time', 'Name', 'Country', 'City', 'Gender', 'Date']]
latest_times

,Time,Name,Country,City,Gender,Date
9,2:02:42,Eliud Kipchoge,KEN,Berlin,Men,2023-09-24
18191,2:11:53,Tigist Assefa,ETH,Berlin,Women,2023-09-24
406,2:05:54,Evans Chebet,KEN,Boston,Men,2023-04-17
18504,2:21:38,Helen Obiri,KEN,Boston,Women,2023-04-17
0,2:00:35,Kelvin Kiptum,KEN,Chicago,Men,2023-08-10
18192,2:13:44,Sifan Hassan,NED,Chicago,Women,2023-08-10
2,2:01:25,Kelvin Kiptum,KEN,London,Men,2023-04-23
18249,2:18:33,Sifan Hassan,NED,London,Women,2023-04-23
2249,2:08:41,Evans Chebet,KEN,New York,Men,2022-06-11
18850,2:23:23,Sharon Lokedi,KEN,New York,Women,2022-06-11


In [74]:
latest_times['Date'] = latest_times['Date'].dt.strftime('%Y-%m-%d')
latest_times.to_json(
    '../data/latest_times.json', 
    orient='records',
    indent=2,
)

AttributeError: Can only use .dt accessor with datetimelike values

In [60]:
people_count = marathons.groupby('City')['Name'].nunique().reset_index(name='People Count')
people_count

,City,People Count
0,Berlin,356
1,Boston,235
2,Chicago,255
3,London,350
4,New York,165
5,Tokyo,309


In [64]:
count_by_gender = marathons.groupby(['City', 'Gender'])\
.agg(unique_count=('Name', 'nunique'))\
.reset_index()\
.rename(columns={'unique_count': 'People Count By Gender'})

gender_count = count_by_gender.pivot(
    index='City', 
    columns='Gender', 
    values='People Count By Gender'
).reset_index()
gender_count.columns.name = None

gender_count

,City,Men,Women
0,Berlin,162,194
1,Boston,100,135
2,Chicago,134,121
3,London,146,204
4,New York,60,105
5,Tokyo,183,126


In [65]:
country_count = marathons.groupby('City')['Country'].nunique().reset_index(name='Country Count')
country_count

,City,Country Count
0,Berlin,34
1,Boston,42
2,Chicago,30
3,London,50
4,New York,29
5,Tokyo,33


In [72]:
record_count = marathons.groupby('City').size().reset_index(name='Record Count')
record_count

,City,Record Count
0,Berlin,452
1,Boston,359
2,Chicago,368
3,London,649
4,New York,292
5,Tokyo,429


In [73]:
marathon_summary = [record_count, people_count, country_count, gender_count]
merged_df = reduce(lambda left, right: pd.merge(left, right, on='City', how='outer'), marathon_summary)

merged_df

,City,Record Count,People Count,Country Count,Men,Women
0,Berlin,452,356,34,162,194
1,Boston,359,235,42,100,135
2,Chicago,368,255,30,134,121
3,London,649,350,50,146,204
4,New York,292,165,29,60,105
5,Tokyo,429,309,33,183,126


In [75]:
merged_df.to_json(
    '../data/marathon_summary.json', 
    orient='records',
    indent=2,
)